In [ ]:
# 필요한 라이브러리 로드
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 데이터 로드하기

In [ ]:
df = pd.read_csv("data/bank.csv")
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

## 학습, 예측 데이터셋 나누기

### 정답값이자 예측해야 될 값

In [ ]:
# label_name 이라는 변수에 예측할 컬럼의 이름을 담습니다.

label_name = "deposit"
label_name

### 문제(feature)와 답안(label)을 나누기

* X, y를 만들어 줍니다.
* X는 feature, 독립변수, 예) 시험의 문제
* y는 label, 종속변수, 예) 시험의 정답

In [ ]:
# X, y를 만들어 줍니다.
X = df.drop(columns=label_name)
y = df[label_name]

In [ ]:
X.head(2)

### 학습, 예측 데이터셋 만들기
* X_train : 학습 세트 만들기, 행렬, 판다스의 데이터프레임, 2차원 리스트(배열) 구조,  예) 시험의 기출문제
* y_train : 정답 값을 만들기, 벡터, 판다스의 시리즈, 1차원 리스트(배열) 구조, 예) 기출문제의 정답
* X_test : 예측에 사용할 데이터세트를 만듭니다. 예) 실전 시험 문제
* y_test : 예측의 정답값 예) 실전 시험 문제의 정답

In [ ]:
X_train = X.sample(frac=0.8, random_state=42)
X_test = X.drop(X_train.index)

X_train.shape, X_test.shape

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
cat_col = X.select_dtypes(include="O").columns
X[cat_col] = X[cat_col].astype("category")

In [ ]:
from sklearn.model_selection import train_test_split

X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [ ]:
X_train_raw.head(1)

In [ ]:
cat_col

In [ ]:
X.select_dtypes(include="category").columns.tolist()

In [ ]:
X.select_dtypes(include="number").columns

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
# column_trans = ColumnTransformer(
#     [('categories', OneHotEncoder(dtype='int'), 
#       X.select_dtypes(exclude="number").columns.tolist())],
#     remainder=MinMaxScaler(), verbose_feature_names_out=False)


# column_trans.fit(X_train)

In [ ]:
column_trans = ColumnTransformer(
    [('categories', OneHotEncoder(dtype='int'), 
      X.select_dtypes(exclude="number").columns.tolist())],
    remainder='passthrough', verbose_feature_names_out=False)

column_trans.fit(X_train)

In [ ]:
pd.DataFrame(column_trans.fit_transform(X_train), 
    columns=column_trans.get_feature_names_out()).describe()

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

ct = make_column_transformer(
      (StandardScaler(),
       make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"),
       make_column_selector(dtype_include="category")))

In [ ]:
X_train = pd.DataFrame(column_trans.fit_transform(X_train_raw), 
    columns=column_trans.get_feature_names_out())

In [ ]:
X_valid = pd.DataFrame(column_trans.fit_transform(X_valid_raw), 
    columns=column_trans.get_feature_names_out())

In [ ]:
X_train.head(2)

## 머신러닝 모델로 예측하기

In [ ]:
# 트리모델 로드하기
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier()
model

In [ ]:
# 학습하기
model.fit(X_train, y_train)

In [ ]:
# 예측하기
y_predict = model.predict(X_valid)
y_predict[:5]

## 모델 평가하기

### 점수 측정하기
#### Accuracy

In [ ]:
# 모델이 얼마나 잘 예측했는지 측정해 봅니다.
# 시험을 보고 나서 몇 개 맞았는지 틀렸는지 채점해 보는 과정과 유사하다고 보면 됩니다.
# 실전 문제에서는 답이 없어서 우리가 예측을 해야겠지만 
# 이 실습은 정답이 있는 모의고사 입니다. 
# 모의고사를 보면 내 실력이 어느정도 되는지 예측해 볼 수 있듯이 점수를 측정해 봅니다.
from sklearn.metrics import accuracy_score

accuracy_score(y_valid, y_predict)

In [ ]:
# 정답과 같이 예측한 값은 True 로 나오게 됩니다. 
# True == 1 이기 때문에 평균값을 내면 정답을 맞춘 비율을 구할 수 있습니다.
(y_valid == y_predict).mean()

#### F1 score
* precision 과 recall의 조화평균
* [정밀도와 재현율 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%A0%95%EB%B0%80%EB%8F%84%EC%99%80_%EC%9E%AC%ED%98%84%EC%9C%A8)

In [ ]:
# 혼동행렬(confusion_matrix)을 그립니다.
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(model, X_train, y_train)

In [ ]:
# 하지만 암환자를 예측한다든지 코로나 양성여부를 판단한다든지 할때는 
# 1000명 검사자 중에 한 두 명만 양성일 수도 있습니다. 
# 이럴 때 1명의 양성은 찾지 못했지만 다른 사람들이 음성인 것을 맞게 예측했다면 정확한 모델이 아님에도 정확도가 높게 나옵니다.
from sklearn.metrics import classification_report

report = classification_report(y_valid, y_predict)
print(report)

In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_estimator(model, X_valid, y_valid)

In [ ]:
# 예측 확률 계산
y_pred_prob = model.predict_proba(X_valid)[:, 1]
y_pred_prob[:5]

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
# ROC 커브 계산
fpr, tpr, thresholds = roc_curve((y_valid == "Yes"), y_pred_prob)

# AUC 계산
roc_auc = auc(fpr, tpr)

# ROC 커브 시각화
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle=':')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()